# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

- Таблица состоит из 21525 строк и 12 столбцов: 2 типа float64, 5 int64 и 5 типа object.  
- Все названия столбцов приведены к snake case.
- 2 столбца (days_employed и total_income) содержат пропуски  
- Есть подозрения относительно корректности типов данных: days_employed - float64 (ожидается int64), total_income - float64 (ожидается int64)

Далее проанализируем каждый из столбцов с точки зрения  
- mean, median;  
- min / max;  
- уникальных значений для нечисловых рядов. 

для первичной оценки качества данных.

In [3]:
#
for column in data:
    print(data[column].name)
    print(f'Всего наблюдений: {data[column].count()}')
    try:
        print(f'mean = {data[column].mean()}')
        print(f'median = {data[column].median()}')
        print(f'min = {min(data[column])}')
        print(f'max = {max(data[column])}')
    except:
        print('Нечисловое поле. Уникальные значения:')
        print(data.groupby(column)[column].count().sort_values(ascending=False))
    print()



children
Всего наблюдений: 21525
mean = 0.5389082462253194
median = 0.0
min = -1
max = 20

days_employed
Всего наблюдений: 19351
mean = 63046.49766147338
median = -1203.369528770489
min = -18388.949900568383
max = 401755.40047533

dob_years
Всего наблюдений: 21525
mean = 43.29337979094077
median = 42.0
min = 0
max = 75

education
Всего наблюдений: 21525
Нечисловое поле. Уникальные значения:
education
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

education_id
Всего наблюдений: 21525
mean = 0.8172357723577236
median = 1.0
min = 0
max = 4

family_status
Всего наблюдений: 21525

In [4]:
# Код реьвюера
data.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,1.935100e+04,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,среднее,NaN,женат / замужем,NaN,F,сотрудник,NaN,NaN,свадьба
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,1.674223e+05,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,1.029716e+05,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,2.066726e+04,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.030532e+05,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.450179e+05,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,2.034351e+05,NaN


**Возможные проблемы в данных**  

*children*
- есть отрицательные значения
- есть аномально высокие (max = 20)

*days_employed*
- есть пропуски
- есть отрицательные значения
- есть аномально высокие (max = 401755)
- медиана < 0

*dob_years*
- есть наблюдения со значением 0

*education*
- наличие дубликатов

*gender*
- наблюдение со значением XNA

*total_income*
- есть пропуски

*purpose*
- есть неявные дубликаты целей

**Следующие шаги**  
<br>
Для дальнейшей работы будем использовать следующую логику: поскольку пропуски будем заменять средними значениями по категориям, необходима первичная предобработка данных:  
1. Разберем переменные с подозрительными значениями: children, dob_years, gender
2. Удалим неявные дубликаты в переменной education
3. Удалим явные дубликаты
4. Заполним пропуски в days_employed и total_income
5. Изменим тип данных переменных days_employed и total_income
6. Лемматизируем столбец purpose
7. Категоризуем переменные
8. Проведем необходимый анализ

## Шаг 2. Предобработка данных

### Обработка переменных с подозрительными данными

Разберем переменные children, dob_years, gender  
  
**children**

In [5]:
print(data.groupby('children')['education'].count())

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: education, dtype: int64


Можно заметить, что в датасете есть 47 наблюдений со значением "-1" и 76 наблюдений со значением "20". Взглянем на них поочередно

In [6]:
display(data[data['children'] == -1].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,NaN,57,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку своего автомобиля
1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


Явной зависимости не наблюдается, будем считать, что вместо "-1" должно стоять значение "1". Перед заменой проанализируем строки со значением "20"

In [7]:
display(data[data['children'] == 20].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,NaN,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,NaN,профильное образование
3396,20,NaN,56,высшее,0,женат / замужем,0,F,компаньон,0,NaN,высшее образование
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,-2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,-805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


Очень маловероятно, что у данных клиентов по 20 детей :) причины могут быть в следующем:  
1. 20 значит 5+ детей
2. 20 - это 2

Первый вариант не кажется возможным, так как в выборку попали клиенты, которым едва за 20 лет, поэтому остановимся на втором варианте и заменим 20 на 2. Ниже произведем две замены, описанные выше и проверим результат. 

In [8]:
def children_correct(value):
    if value == -1:
        return 1
    elif value == 20:
        return 2
    else:
        return value

data['children'] = data['children'].apply(children_correct)
data.reset_index(drop=True)

print(data.groupby('children')['children'].count())
        

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


**dob_years**  
Разберем наблюдения со значением "0"

In [9]:
print(data[data['dob_years'] == 0]['dob_years'].count())
display(data[data['dob_years'] == 0].head(10))

101


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


Всего таких 101 наблюдение, явной причины отсутствия возраста также нет. Поскольку 101 наблюдение составляет 0.5% от общей выборки, эти данные можно исключить из дальнейшего анализа

In [10]:
data = data[data['dob_years'] != 0]
data.reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21419,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21420,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21421,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21422,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**gender**  
В данной переменной у нас есть 1 наблюдение со значением XNA, взглянем на него:

In [11]:
display(data[data['gender'] == 'XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Явных признаков, по которым можно определить пол, здесь нет, поэтому имеет смысл исключить это наблюдение.

In [12]:
data = data[data['gender'] != 'XNA']
data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
children            21423 non-null int64
days_employed       19259 non-null float64
dob_years           21423 non-null int64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
debt                21423 non-null int64
total_income        19259 non-null float64
purpose             21423 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Обработка дубликатов

Сначала избавимся от неявных дубликатов в переменной education. Судя по регистру, поле имеет ручной ввод, что привело к их появлению. Приведем все поля к нижнему регистру.

In [13]:
data['education'] = data['education'].str.lower()
print(data.groupby('education')['education'].count())

education
высшее                  5225
начальное                282
неоконченное высшее      741
среднее                15169
ученая степень             6
Name: education, dtype: int64


Далее проанализируем число явных дубликатов

In [14]:
print(data.duplicated().sum())

71


В датасете присутствует 71 дубликат, удалим эти наблюдения.

In [15]:
data = data.drop_duplicates().reset_index(drop=True)
data.info()
print(data.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21352 entries, 0 to 21351
Data columns (total 12 columns):
children            21352 non-null int64
days_employed       19259 non-null float64
dob_years           21352 non-null int64
education           21352 non-null object
education_id        21352 non-null int64
family_status       21352 non-null object
family_status_id    21352 non-null int64
gender              21352 non-null object
income_type         21352 non-null object
debt                21352 non-null int64
total_income        19259 non-null float64
purpose             21352 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
0


### Обработка пропусков

In [16]:
data.isnull().sum()
data.isna().sum()

children               0
days_employed       2093
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2093
purpose                0
dtype: int64

Пропуски содержатся в двух столбцах: days_employed и total_income.<br>

**days_employed**<br>
Заменим пропущенные значения на средние в группах income_type -> dob_years. Логика такова, что стаж примерно равен в группах в зависимости от типа занятости и возраста.  
Для корректного подсчета средних необходимо избавиться от отрицательных значений и понять природу данных (почему присутствуют такие большие значения). Для этого сначала посчитаем число отрицательных:

In [17]:
print(data[data['days_employed'] < 0]['days_employed'].count())

15831


3/4 значений стажа - отрицательные. Предположим, что это ошибка в знаке и изменим значения на положительные.

In [18]:
data['days_employed'] = abs(data['days_employed'])

Посчитаем среднее и медиану получившегося ряда по основным категориальным переменным:

In [19]:
categories = ['education_id', 'family_status_id', 'income_type']
for cat in categories:
    print(data.groupby(cat)['days_employed'].mean())  
    print()

education_id
0     42352.124485
1     76376.801662
2     20744.618019
3    130340.426349
4    121323.630206
Name: days_employed, dtype: float64

family_status_id
0     63305.475668
1     58685.455954
2    204893.316311
3     69002.258467
4     46764.024176
Name: days_employed, dtype: float64

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          3388.508552
компаньон            2112.744402
пенсионер          365015.727554
предприниматель       520.848083
сотрудник            2328.603723
студент               578.751554
Name: days_employed, dtype: float64



Можно заметить, что среднее внутри групп "безработный" и "пенсионер" составляет 1000 лет и сильно смещает среднее общей выборки.
Проанализируем min и max внутри группы, чтобы понять разбег значений.

In [20]:
print('Max - ', data[data['income_type'] == 'пенсионер']['days_employed'].max())
print('Min - ', data[data['income_type'] == 'пенсионер']['days_employed'].min())

Max -  401755.40047533
Min -  328728.72060451825


Если предположить, что данные по группе занесены в часах, то  
Max ~ 46 лет   
Min ~ 38 лет  
что похоже на фактический стаж данной группы. Приведем для данной группы часы к дням:

In [21]:
for index in range(len(data)):
    if data['income_type'][index] == 'пенсионер':
        data['days_employed'][index] = data['days_employed'][index] / 24

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Проверим обновленное среднее по группам:

In [22]:
print(data.groupby('income_type')['days_employed'].mean())

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          3388.508552
компаньон            2112.744402
пенсионер           15208.988648
предприниматель       520.848083
сотрудник            2328.603723
студент               578.751554
Name: days_employed, dtype: float64


В статусе "безработный" у нас находится всего 2 наблюдения, рабочий стаж в которых указан не верно. Удалим эти 2 наблюдения из датасета, чтобы не смещать выборку.

In [23]:
data = data[data['income_type'] != 'безработный']
data.reset_index(drop=True)
print(data.groupby('income_type')['days_employed'].mean())

income_type
в декрете           3296.759962
госслужащий         3388.508552
компаньон           2112.744402
пенсионер          15208.988648
предприниматель      520.848083
сотрудник           2328.603723
студент              578.751554
Name: days_employed, dtype: float64


Создадим категории в зависимости от возраста:  
- 25-
- 26-35
- 36-45
- 46-55
- 56-65
- 65+

In [24]:
def age_group(age):
    if age <= 25:
        return '25-'
    elif age <= 35:
        return '26-35'
    elif age <= 45:
        return '36-45'
    elif age <= 55:
        return '46-55'
    elif age <= 65:
        return '56-65'
    else:
        return '65+'
    
data['age_group'] = data['dob_years'].apply(age_group)
data_pivot = data.pivot_table(index=['income_type', 'age_group'], values='education_id', aggfunc='count')
display(data_pivot)

education_id
income_type     age_group              
в декрете       36-45                 1
госслужащий     25-                  73
                26-35               432
                36-45               458
                46-55               346
                56-65               133
                65+                   9
компаньон       25-                 378
                26-35              1546
                36-45              1587
                46-55              1145
                56-65               374
                65+                  27
пенсионер       25-                   2
                26-35                16
                36-45                84
                46-55               796
                56-65              2297
                65+                 614
предприниматель 26-35                 1
                56-65                 1
сотрудник       25-                 777
                26-35              3355
                36-45              3489
                46-55              2527
                56-65               829
                65+                  52
студент         25-                   1

Из получившегося распределения видно, что:  
- категория "пенсионеры" содержит людей моложе 45 лет, чего быть не может  
- категории "в декрете", "студент", "предприниматель" содержат по 1-2 наблюдения.  
  
Для получения более объективных результатов удалим эти категории из датасета и выведем новые средние стажа по категориям.

In [25]:
data = data[data['income_type'] != 'в декрете']
data = data[data['income_type'] != 'студент']
data = data[data['income_type'] != 'предприниматель']
index_names = data[(data['income_type'] == 'пенсионер') & (data['dob_years'] <= 45)].index
data.drop(index_names, inplace = True)
data.reset_index(drop=True)
data_pivot = data.pivot_table(index=['income_type', 'age_group'], values='days_employed', aggfunc='mean')
display(data_pivot)

days_employed
income_type age_group               
госслужащий 25-          1174.478880
            26-35        2123.168852
            36-45        3701.579316
            46-55        4438.451507
            56-65        4936.760900
            65+          4145.742201
компаньон   25-           860.317166
            26-35        1579.732861
            36-45        2172.231307
            46-55        2791.876907
            56-65        3159.044857
            65+          3725.387000
пенсионер   46-55       15184.411127
            56-65       15212.555051
            65+         15235.018791
сотрудник   25-           930.982057
            26-35        1671.382722
            36-45        2456.651823
            46-55        3023.302599
            56-65        3538.604796
            65+          4092.413329

Далее заменим пропуски в данных по стажу на соответствующие средние внутри категорий.

In [26]:
data = data.reset_index(drop=True)
data['days_employed'] = data['days_employed'].fillna('unknown')
for index in range(len(data)):
    if data['days_employed'][index] == 'unknown':
        a = data['income_type'][index]
        b = data['age_group'][index]
        data['days_employed'][index] = data_pivot.loc[a,b]
data.info()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21244 entries, 0 to 21243
Data columns (total 13 columns):
children            21244 non-null int64
days_employed       21244 non-null object
dob_years           21244 non-null int64
education           21244 non-null object
education_id        21244 non-null int64
family_status       21244 non-null object
family_status_id    21244 non-null int64
gender              21244 non-null object
income_type         21244 non-null object
debt                21244 non-null int64
total_income        19167 non-null float64
purpose             21244 non-null object
age_group           21244 non-null object
dtypes: float64(1), int64(5), object(7)
memory usage: 2.1+ MB


**total_income**<br>
Ряд total_income также содержит пропуски. Для данного ряда заменим пропуски на средние в категориях   
income_type > age_group > education, предполагая, что доход зависит от типа занятости, возраста и образования.

In [27]:
data_pivot_income = data.pivot_table(index=['income_type', 'age_group', 'education'], values='total_income', aggfunc='mean')
display(data_pivot_income)

total_income
income_type age_group education                         
госслужащий 25-       высшее               177064.770041
                      неоконченное высшее  130568.327456
                      среднее              144045.207619
            26-35     высшее               183780.440129
                      начальное            191021.140719
...                                                  ...
сотрудник   56-65     ученая степень       268411.214536
            65+       высшее               180572.091228
                      начальное            179369.498314
                      неоконченное высшее  211207.224592
                      среднее              155919.466573

[83 rows x 1 columns]

Заменим пропуски на соответствующие средние по категориям.

In [28]:
data['total_income'] = data['total_income'].fillna('unknown')

for index in range(len(data)):
    if data['total_income'][index] == 'unknown':
        a = data['income_type'][index]
        b = data['age_group'][index]
        c = data['education'][index]
        data['total_income'][index] = data_pivot_income.loc[a,b,c]
data.info()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21244 entries, 0 to 21243
Data columns (total 13 columns):
children            21244 non-null int64
days_employed       21244 non-null object
dob_years           21244 non-null int64
education           21244 non-null object
education_id        21244 non-null int64
family_status       21244 non-null object
family_status_id    21244 non-null int64
gender              21244 non-null object
income_type         21244 non-null object
debt                21244 non-null int64
total_income        21244 non-null object
purpose             21244 non-null object
age_group           21244 non-null object
dtypes: int64(5), object(8)
memory usage: 2.1+ MB


### Замена типа данных

Следуя пункту №1, замена типа данных требуется двум переменным: days_employed и total_income. Сменим тип на целочисленный методом astype. Для анализа стажа и общего уровня дохода нам не интересны знаки после запятой, так как они не несут в себе смысла.

In [29]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21244 entries, 0 to 21243
Data columns (total 13 columns):
children            21244 non-null int64
days_employed       21244 non-null int64
dob_years           21244 non-null int64
education           21244 non-null object
education_id        21244 non-null int64
family_status       21244 non-null object
family_status_id    21244 non-null int64
gender              21244 non-null object
income_type         21244 non-null object
debt                21244 non-null int64
total_income        21244 non-null int64
purpose             21244 non-null object
age_group           21244 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


### Лемматизация

In [30]:
print(data.groupby('purpose')['purpose'].count())

purpose
автомобили                                472
автомобиль                                486
высшее образование                        448
дополнительное образование                458
жилье                                     637
заняться высшим образованием              488
заняться образованием                     405
на покупку автомобиля                     464
на покупку подержанного автомобиля        477
на покупку своего автомобиля              501
на проведение свадьбы                     756
недвижимость                              626
образование                               442
операции с жильем                         645
операции с коммерческой недвижимостью     644
операции с недвижимостью                  667
операции со своей недвижимостью           625
покупка жилой недвижимости                602
покупка жилья                             638
покупка жилья для сдачи                   643
покупка жилья для семьи                   634
покупка коммерческой недви

**Вывод**

Если просмотреть весь список, то можно заметить, что все цели получения кредита можно свести к четырем основным:  
- автомобиль
- недвижимость
- образование
- свадьба  

Лемматизируем цели

In [31]:
from pymystem3 import Mystem
m = Mystem()

def lemmatize_text(text):
    return m.lemmatize(text)

data['purpose'] = data['purpose'].apply(lemmatize_text)

print(data['purpose'])

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21239                  [операция,  , с,  , жилье, \n]
21240               [сделка,  , с,  , автомобиль, \n]
21241                              [недвижимость, \n]
21242    [на,  , покупка,  , свой,  , автомобиль, \n]
21243             [на,  , покупка,  , автомобиль, \n]
Name: purpose, Length: 21244, dtype: object


### Категоризация данных

Для дальнейшего анализа нам потребуется следующие категории:  
- наличие детей
- уровни дохода
- цели кредита

**children**

In [32]:
def children_cat(value):
    if value > 0:
        return 'Есть дети'
    else:
        return 'Нет детей'
    
data['children_cat'] = data['children'].apply(children_cat)
print(data.groupby('children_cat')['children_cat'].count())

children_cat
Есть дети     7294
Нет детей    13950
Name: children_cat, dtype: int64


**income**  
Выделим категории по следующему принципу:  
1. min - mediana/2: низкий
2. mediana/2 - mediana: ниже среднего
3. mediana - 3/2mediana: выше среднего
4. 3/2mediana - max: высокий



In [33]:
minimum = min(data['total_income'])
median = data['total_income'].median()
med_1 = median/2
med_3 = median/2 * 3
maximum = max(data['total_income'])

print(minimum,med_1,median,med_3,maximum)

def income_cat(value):
    if (value >= minimum) & (value < med_1):
        return 'низкий'
    elif (value >= med_1) & (value < median):
        return 'ниже среднего'
    elif (value >= median) & (value < med_3):
        return 'выше среднего'
    else:
        return 'высокий'
    
data['total_income_cat'] = data['total_income'].apply(income_cat)
print(data.groupby('total_income_cat')['total_income_cat'].count())

20667 75154.0 150308.0 225462.0 2265604
total_income_cat
высокий          3904
выше среднего    6844
ниже среднего    8649
низкий           1847
Name: total_income_cat, dtype: int64


**purpose**  
Распределим лемматизированную цель по 4 категориям: автомобиль, недвижимость, образование, свадьба

In [34]:
def purpose_cat(lis):
    if 'автомобиль' in lis:
        return 'автомобиль'
    elif ('недвижимость' in lis) or ('жилье' in lis):
        return 'недвижимость'
    elif 'образование' in lis:
        return 'образование'
    else:
        return 'свадьба'
    
data['purpose_cat'] = data['purpose'].apply(purpose_cat)
print(data.groupby('purpose_cat')['purpose_cat'].count())

purpose_cat
автомобиль       4266
недвижимость    10714
образование      3976
свадьба          2288
Name: purpose_cat, dtype: int64


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
print(data.groupby('children_cat')['debt'].mean())

children_cat
Есть дети    0.091582
Нет детей    0.075341
Name: debt, dtype: float64


**Вывод**

Средний процент невозврата кредита выше у клиентов с детьми, чем у клиентов без детей (9.1% vs 7.5% соответственно). Отчасти, это может объясняться постоянными непредвиденными расходами на детей, которые могут смещать сроки выплат.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [36]:
print(data.groupby('family_status')['debt'].mean().sort_values(ascending=False))

family_status
Не женат / не замужем    0.097736
гражданский брак         0.092972
женат / замужем          0.075188
в разводе                0.072218
вдовец / вдова           0.064211
Name: debt, dtype: float64


**Вывод**

Самый высокий процент невозврата у групп со статусом "не женат" и "гражданский брак". Можно предположить, что в данные группы попадают более молодые люди с менее стабильным заработком, что подтверждается и на данных:

In [37]:
print(data.groupby('family_status')['dob_years'].mean().sort_values())

family_status
Не женат / не замужем    38.596838
гражданский брак         42.314300
женат / замужем          43.735453
в разводе                45.961767
вдовец / вдова           56.871579
Name: dob_years, dtype: float64


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [38]:
print(data.groupby('total_income_cat')['debt'].mean().sort_values(ascending=False))

total_income_cat
ниже среднего    0.085443
выше среднего    0.083869
низкий           0.072009
высокий          0.069928
Name: debt, dtype: float64


**Вывод**

Процент невозврата наиболее высокий у слоев населения со средним доходом, при этом дисциплина выплат у клиентов с низким и высоким уровнем на порядок выше.  Возможно, это связано с тем, что  
- для обеспеченных клиентов выплатить кредит в срок не проблема
- для малообеспеченных обязательства перед банком составляют важную часть их жизни, в том числе влияя в дальнейшем на кредитную историю, которой им лучше не пренебрегать.

- Как разные цели кредита влияют на его возврат в срок?

In [39]:
print(data.groupby('purpose_cat')['debt'].mean().sort_values(ascending=False))

purpose_cat
автомобиль      0.092827
образование     0.092304
свадьба         0.078671
недвижимость    0.072429
Name: debt, dtype: float64


**Вывод**

Наиболее уязвимые категории с точки зрения невозврата: автомобиль и образование, при этом более "фундаментальные" для людей цели кредитования в виде недвижимости и свадьбы несут в себе меньшие риски.

## Шаг 4. Общий вывод

Исходя из получившихся результатов, можно сделать следующие выводы:
- чем более фундаментальна цель кредитования, тем ниже % невозврата
- клиенты, находящиеся в браке или прошедшие его, более дисциплинированы
- наличие детей негативно влияет на оперативность возвращения кредита
- клиенты со средним уровнем заработка - наименее дисциплинированные

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.